In [1]:
%matplotlib widget
from typing import Callable, Tuple
import functools
import numpy as np
import matplotlib.pyplot as plt

# Some useful utilities
from mcmc_utils_and_plot import scatter_matrix, build_cov_mat, lognormpdf, plot_bivariate_gauss, eval_func_on_grid

In [2]:
# Normal Metropolis Hastings

Log_pdf_t = Callable[[np.ndarray], float]  # log pdf type
Log_pdfc_t = Callable[[np.ndarray, np.ndarray], float]  # conditional pdf types
Sampler = Callable[[np.ndarray], np.ndarray]  # conditional sampler type


def mh_acceptance_prob(current_target_logpdf: float,
                       proposed_target_logpdf: float,
                       current_sample: np.ndarray,
                       proposed_sample: np.ndarray,
                       proposal_func: Log_pdfc_t) -> float:
    """Compute the metropolis-hastings accept-reject probability.

    Args:
        current_target_logpdf: float, logpdf at the current sample in the chain f_X(x^{(k)})
        proposed_target_logpdf: float, logpdf at the proposed sample in the chain
        current_sample: (d, ), current sample
        proposed_sample: (d, ), proposed sample
        proposal_func: f(x, y) callable that gives the log probability of y given x

    Returns:
        acceptance probability
    """

    prop_reverse = proposal_func(proposed_sample, current_sample)
    prop_forward = proposal_func(current_sample, proposed_sample)
    check = proposed_target_logpdf - current_target_logpdf + prop_reverse - prop_forward  # check this
    if check < 0:
        return np.exp(check)
    return 1.0


def mh(starting_sample: np.ndarray,
       num_samples: int,
       target_logpdf: Log_pdf_t,
       proposal_logpdf: Log_pdfc_t,
       proposal_sampler: Sampler) -> Tuple[np.ndarray, float]:
    """Metropolis-Hastings MCMC.

    Args:
        starting_sample: (d, ) the initial sample
        num_sample: positive integer, the number of total samples
        target_logpdf: function(x) -> logpdf of the target distribution
        proposal_logpdf: function (x, y) -> logpdf of proposing y if current sample is x
        proposal_sampler: function (x) -> y, generate a sample if you are currently at x

    Returns:
        Samples: (num_samples, d) array of samples
        accept_ratio: ratio of proposed samples that were accepted
    """

    d = starting_sample.shape[0]
    samples = np.zeros((num_samples, d))
    samples[0, :] = starting_sample
    current_target_logpdf = target_logpdf(samples[0, :])

    num_accept = 0
    for ii in range(1, num_samples):
        # propose
        proposed_sample = proposal_sampler(samples[ii-1, :])
        proposed_target_logpdf = target_logpdf(proposed_sample)

        # determine acceptance probability
        a = mh_acceptance_prob(current_target_logpdf, proposed_target_logpdf,
                               samples[ii-1, :], proposed_sample, proposal_logpdf)

        # Accept or reject the sample
        if a == 1:  # guaranteed to accept
            samples[ii, :] = proposed_sample
            current_target_logpdf = proposed_target_logpdf
            num_accept += 1
        else:
            u = np.random.rand()
            if u < a:  # accept
                samples[ii, :] = proposed_sample
                current_target_logpdf = proposed_target_logpdf
                num_accept += 1
            else:  # reject
                samples[ii, :] = samples[ii-1, :]

    return samples, num_accept / float(num_samples-1)

# 1 Adaptive Metropolis Hastings
# 2. Delayed Rejection
# 3. Delayed Rejection Adaptive Metropolis

In [3]:
# 4. Target (Banana distribution)
def banana_logpdf(x):
    a = 1.0
    b = 100.0
    logpdf = -(a-x[0])**2 - b * (x[1] - x[0]**2)**2
    # logpdf = np.exp(-0.5 * (x[0])**2 + x[1]**2) - np.exp(-0.5 / 1.0 * (x[0]**2 + x[1]**2))
    # logpdf = np.log(logpdf)
    # logpdf = (np.sin(10*x[0]*x[1]) + x[1]**2)*4
    return logpdf
    # return logpdf
# Proposal functions


def proposal_rw_sampler(x: np.ndarray, std: float = 1.0) -> np.ndarray:
    """Sample from a random walk proposal with identity covariance."""
    y = std * np.random.randn(x.shape[0]) + x
    return y


def proposal_rw_logpdf(x: np.ndarray, y: np.ndarray, std: float = 1.0) -> np.ndarray:
    """Probability of moving from x to y (in this case it is symmetric)."""
    delta = x - y
    logpdf = -0.5 * np.dot(delta, delta) / std / std
    return logpdf

In [4]:
# Evaluate all the sampling methods

num_samples = 100000
dim = 2  # do not change cos banana is only defined for 2D
initial_sample = np.random.randn(dim)  # random location
# initial_sample = np.array([10, 10])

target_logpdf = banana_logpdf


def p_sampler(x): return proposal_rw_sampler(x, std=3e-1)  # 3e-1
def p_eval(x, y): return proposal_rw_logpdf(x, y, std=3e-1)


s_mh, ar_mh = mh(initial_sample, num_samples, target_logpdf, p_eval, p_sampler)
# s_am, ar_am = am(initial_sample, num_samples, target_logpdf, p_eval, p_sampler)
# s_dr, ar_dr = dr(initial_sample, num_samples, target_logpdf, p_eval, p_sampler)
# s_dram, ar_dram = dram(initial_sample, num_samples, target_logpdf, p_eval, p_sampler)

In [5]:
# Analysis functions

def autocorrelation(samples: np.ndarray, maxlag: int = 100, step: int = 1):
    """Compute the correlation of a set of samples.

    Args:
        samples: (N, d)
        maxlag: maximum distance to compute the correlation for
        step: step between distances from 0 to maxlag for which to compute the correlations
    """

    # Get the shapes
    ndim = samples.shape[1]
    nsamples = samples.shape[0]

    # Compute the mean
    mean = np.mean(samples, axis=0)

    # Compute the denominator, which is variance
    denominator = np.zeros((ndim))
    for ii in range(nsamples):
        denominator = denominator + (samples[ii, :] - mean)**2

    lags = np.arange(0, maxlag, step)
    autos = np.zeros((len(lags), ndim))
    for zz, lag in enumerate(lags):
        autos[zz, :] = np.zeros((ndim))
        # compute the covariance between all samples *lag apart*
        for ii in range(nsamples - lag):
            autos[zz, :] = autos[zz, :] + (samples[ii, :]-mean)*(samples[ii + lag, :] - mean)
        autos[zz, :] = autos[zz, :]/denominator
    return lags, autos


def autocorr_fast(samples: np.ndarray, maxlag: int = 100, step: int = 1):
    """Compute the correlation of a set of samples.

    Args:
        samples: (N, d)
        maxlag: maximum distance to compute the correlation for
        step: step between distances from 0 to maxlag for which to compute the correlations
    """

    # Get the shapes
    ndim = samples.shape[1]

    # Compute the mean
    mean = np.mean(samples, axis=0)

    denominator = np.sum((samples - mean)**2, axis=0)

    lags = np.arange(0, maxlag, step)
    autos = np.zeros((len(lags), ndim))
    autos[0, :] = denominator * np.ones((ndim))  # lag 0 has max autocorrelation of 1
    for zz, lag in enumerate(lags[1:]):  # skip the lag of 0
        autos[zz+1, :] = np.sum((samples[:-lag, :] - mean) * (samples[lag:, :] - mean), axis=0)
    autos = autos/denominator
    return lags, autos


def autocorr_test(samples: np.ndarray, maxlag: int = 100, step: int = 1):
    lags1, autolag1 = autocorrelation(samples, maxlag=maxlag, step=step)
    lags2, autolag2 = autocorr_fast(samples, maxlag=maxlag, step=step)
    assert np.allclose(autolag1, autolag2), "Autocorrelation functions are not the same"
    assert np.allclose(lags1, lags2), "Lags are not the same"


def get_iacs(sampless):
    iacs = []
    for samples in sampless:
        maxlag = samples.shape[0]
        _, autolag = autocorr_fast(samples, maxlag=maxlag, step=1)
        iacs.append(1 + 2*np.sum(autolag, axis=0))
    return iacs

In [6]:
# autocorr_test(s_mh)

KeyboardInterrupt: 

In [ ]:
# Analysis
# samples = [s_mh, s_am, s_dr, s_dram]
sampless = [s_mh]

# titles = ["Metropolis Hastings", "Adaptive Metropolis",
#             "Delayed Rejection", "Delayed Rejection Adaptive Metropolis"]
titles = ["Metropolis Hastings"]

# Autocorrelation

MAXLAG = 1000
STEP = 10

lags_auto = []
autolags = []
for samples in sampless:
    lags, autolag = autocorr_fast(samples, maxlag=MAXLAG, step=STEP)
    lags_auto.append(lags)
    autolags.append(autolag)

In [ ]:
iacs = get_iacs(sampless)

In [ ]:
# Plot Helper functions


def plot_marginal(plt_id, sampless, titles):
    for samples, title in zip(sampless, titles):
        plt.close(plt_id)
        fig, _, _ = scatter_matrix(plt_id, [samples], labels=[r'$x_1$', r'$x_2$'],
                                   hist_plot=True, gamma=0.2, nbins=70, title=f'Marginal: {title}')
        plt.show()
        fig.savefig(f'figs/marginal_{title}.svg')
        plt_id += 1

    return plt_id


def plot_autocorrelation(plt_id, lagss, autolags, titles):
    n_plots = len(lagss)
    plt.close(plt_id)
    fig, axs = plt.subplots(n_plots, 2, figsize=(8, n_plots*2.5), num=plt_id)
    fig.tight_layout(pad=3)
    fig.suptitle('Autocorrelation', fontsize=14)
    for i, (lags, autolag, title) in enumerate(zip(lagss, autolags, titles)):
        # ax1 = axs[i*2, 0]
        # ax2 = axs[i*2, 1]
        ax1 = axs[0]
        ax2 = axs[1]
        ax1.plot(lags, autolag[:, 0], '-o')
        ax1.set_xlabel('Lag')
        ax1.set_ylabel('Autocorrelation')
        ax1.set_title(title + r" $x_1$")
        ax2.plot(lags, autolag[:, 1], '-o')
        ax2.set_xlabel('Lag')
        ax2.set_ylabel('Autocoorelation')
        ax2.set_title(title + r" $x_2$")
    plt.show()
    fig.savefig('figs/autocorrelation.svg')
    plt_id += 1
    return plt_id


def plot_inspection(plt_id, sampless, titles):
    n_plts = len(sampless)
    plt.close(plt_id)
    fig, axs = plt.subplots(n_plts, 2, figsize=(8, n_plts*2.5), num=plt_id)
    fig.tight_layout(pad=3)
    fig.suptitle('Visual Inspection of Mixing', fontsize=14)
    for i, (samples, title) in enumerate(zip(sampless, titles)):
        # ax1 = axs[i*2, 0]
        # ax2 = axs[i*2, 1]
        ax1 = axs[0]
        ax2 = axs[1]
        ax1.plot(samples[:, 0], '-k')
        ax1.set_xlabel('Sample Number')
        ax1.set_ylabel('Value')
        ax1.set_title(title + r" $x_1$")
        ax2.plot(samples[:, 1], '-k')
        ax2.set_xlabel('Sample Number')
        ax2.set_ylabel('Value')
        ax2.set_title(title + r" $x_2$")
    plt.show()
    fig.savefig('figs/inspection.svg')
    plt_id += 1
    return plt_id

In [ ]:
# 5. Plot everything
plt_id = 1
# 5.a Plots of 1D and 2D marginals
plt_id = plot_marginal(plt_id, sampless, titles)

# 5.b Autocorrelation plots
plt_id = plot_autocorrelation(plt_id, lags_auto, autolags, titles)

# 5.c Integrated autocorrelation values
print(f"Integrated autocorrelation values:")
for iac, title in zip(iacs, titles):
    print(f"- {title}: {iac}")

# 5.d Acceptance ratio
print(f"Acceptance ratio:")
print(f"- Metropolis Hastings: {ar_mh*100:.0f}%")
# print(f"- Adaptive Metropolis: {ar_am*100:.0f}%")
# print(f"- Delayed Rejection: {ar_dr*100:.0f}%")
# print(f"- Delayed Rejection Adaptive Metropolis: {ar_dram*100:.0f}%")

# 5.e Visual inspection of mixing
plt_id = plot_inspection(plt_id, sampless, titles)